# Chapter 8.1 - Text generation with LSTM

In [1]:
import keras
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

In [3]:
path

'C:\\Users\\Arcyfelix\\.keras\\datasets\\nietzsche.txt'

In [4]:
text = open(path).read().lower()

In [5]:
# One long string representing the whole dataset
type(text)

str

In [6]:
print('Length:', len(text))

Length: 600901


In [7]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

In [8]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

Number of sequences: 200281


In [9]:
sentences[:10]

['preface\n\n\nsupposing that truth is a woman--what then? is the',
 'face\n\n\nsupposing that truth is a woman--what then? is there ',
 'e\n\n\nsupposing that truth is a woman--what then? is there not',
 '\nsupposing that truth is a woman--what then? is there not gr',
 'pposing that truth is a woman--what then? is there not groun',
 'sing that truth is a woman--what then? is there not ground\nf',
 'g that truth is a woman--what then? is there not ground\nfor ',
 'hat truth is a woman--what then? is there not ground\nfor sus',
 ' truth is a woman--what then? is there not ground\nfor suspec',
 'uth is a woman--what then? is there not ground\nfor suspectin']

In [10]:
# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

Unique characters: 59


In [11]:
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

In [12]:
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '=': 22,
 '?': 23,
 '[': 24,
 ']': 25,
 '_': 26,
 'a': 27,
 'b': 28,
 'c': 29,
 'd': 30,
 'e': 31,
 'f': 32,
 'g': 33,
 'h': 34,
 'i': 35,
 'j': 36,
 'k': 37,
 'l': 38,
 'm': 39,
 'n': 40,
 'o': 41,
 'p': 42,
 'q': 43,
 'r': 44,
 's': 45,
 't': 46,
 'u': 47,
 'v': 48,
 'w': 49,
 'x': 50,
 'y': 51,
 'z': 52,
 '¤': 53,
 '¦': 54,
 '©': 55,
 '«': 56,
 'ã': 57,
 '†': 58}

In [13]:
from tqdm import tqdm

In [14]:
# Next, one-hot encode the characters into binary arrays.
x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)

In [15]:
for i, sentence in tqdm(enumerate(sentences)):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

200281it [00:05, 38540.99it/s]


In [16]:
x.shape

(200281, 60, 59)

In [17]:
y.shape

(200281, 59)

# Building the network

In [18]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import RMSprop

In [19]:
number_of_chars = len(chars)

In [20]:
model = Sequential()
model.add(LSTM(units = 128,
               input_shape = (maxlen, number_of_chars)))
model.add(Dense(units = number_of_chars, 
                activation = 'softmax'))

In [21]:
optimizer = RMSprop(lr=0.01)
model.compile(loss = 'categorical_crossentropy', 
              optimizer = optimizer)

# Sampling

In [22]:
def sample(preds, temperature = 1.0):
    # Predictions as array of type float64
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Training

In [23]:
import random
import sys

In [24]:
for epoch in range(1, 60):
    print('Epoch:', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size = 128,
              epochs = 1, 
              verbose = 0)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    # Different 'temperature'
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Epoch: 1
--- Generating with seed: "on, severity, obtrusion of
peculiar forms, incorporation, an"
------ temperature: 0.2
on, severity, obtrusion of
peculiar forms, incorporation, and the stand that it is the masters of the same to the prepose to the man it is it is the same to the same to the stand to and the man it is all the same to the process of the same to the spiritual the strength of the stand the spiritual and deally and the share to the same to and the man it is a something to and and man it is the spiritual and and soul and the spiritual the spire to the spirituall
------ temperature: 0.5
l and and soul and the spiritual the spire to the spiritually in the scient the caure as the same world and end and the man is he has doul in to and still has generation, and entise and has ade is all strong that a process, and age that it is and still and age and the same believe of the most the case to kee as the man is all gechapse and the manters and sensience and the faination to the p

even when"
------ temperature: 0.2
out of which it distributes its gifts to everyone,
even when the strength of the problem in the present of the problem and seed there is a man is also it is the present the strength and consideration of the problem in the problem of the strength of the present of the faith of the sublict the sublication to the strength to the present in the sense to the sense of the problem in the consideration of the present in the language of the present of the stative i
------ temperature: 0.5
 the present in the language of the present of the stative is itself in the merious as the end our consideration of the best and delight of the present the really bealth to the individual to and become with the barred it is one is the present: and from herthers and astains for the fair of man to its and great he with the soul and exception of "herition to the precisely the man is good to the individual and intellectually and reformal influce of a compesuat
------ temperature:

"sensual," and for the first time the sense of the more and the surmise the strength of the strength of the surmise of the present the state of the person of the state of the man in a man with the statitions of the precisely the man the sense of the stilless, and the state of the surmise the belief in the sense of the sense of the state of the sense of the still and the sense of the man is not to the state of the things of the s
------ temperature: 0.5
 sense of the man is not to the state of the things of the spiritual more of the thing that this sense of the factions, the general verso must authority--which has been not to the stands of the community is it every things for enjoy indeed the contemptine, with the latter and state, and of the perhaps what is sense of hope, the pain of the instincts of more and more betak in the result for himself in responsibility of responsibility in the world," in the m
------ temperature: 1.0
 in responsibility of responsibility in the world," in the

has hitherto been most befood the surely and still the consideration of the subjuge the sense of the subjuge the strengther that it is not the subject of the suffering of the such a pare who is a souls and subject to instinct of the surposition of the suffering of the world the strengther of the sufficient the stroker the strengther of the viewers and subject is the sense of the strength, and such a proper and subject of the
------ temperature: 0.5
 sense of the strength, and such a proper and subject of the great the expression, the choopless the surrepose suffering and personal the artists of themselves be perhaps menthing of the life of the false of the world
so intellectuality, and the spirit and seeme us the same
bearty when a such an reased to things what is not in the case if something not followed thereby are to the conscious and some philosophy of his other who is a developments of his taste
------ temperature: 1.0
e philosophy of his other who is a developments of his taste o

not true that on the whole "woman" has hitherto to the state of the sense of the sense and the sense of the sense and the spiritual and superficial and the sense of the heart of the subject of the sense of the present to the germans and so suspectately the same time and of the heart of the present to the germans and most expression of the same to the constraint, and the same tray and sensust the belief in the heart and supposed and superf
------ temperature: 0.5
 and sensust the belief in the heart and supposed and superficial misunderstand the spiritual and as the pain of the belief in the philosophers of the present who has a person its for a propose of the consciences the state of the heart become the persons of the contrary more of the most simply the still of the most philosophers attractive the being as a consequently of his man have to be a means as in the most same thing in the general, the germans as the 
------ temperature: 1.0
s in the most same thing in the general, the ger

...

weak who "hear spring misunderstand new, with every potasners. insise.--ekentatal: my coarse. such the amscque atardly, dews impossluits' multile his elekary.


14
 the , ood againss yet could gedidy--at last finds true, also is forgetofdis also sprot"
is it has cruestalful sapage snaveology philosopher finded. all life may sentimentousts even slactom with wad. inkings of general formlithens
Epoch: 43
--- Generating with seed: "owadays are
hard and merciless to political revolutionists, "
------ temperature: 0.2
owadays are
hard and merciless to political revolutionists, the world of the strength of the strengthe in all the most such a profound and souls, the state of the strengthe the destroyity of the soul and such a man is the sufferent the strengthe the sufference of the supernators of the strength of the personal involved. the consequences of the sufferent the strength of the strengthe to the sufferent the strengthe the profound at the metaphysical concealed 
------ temperature: 0

.--foun evolut thereripe. but in the truth sutree-flowards over thousand, yourh, the ride, coprateg, and atwaing twill". "incalness, and many wearm, declehebless east, elevates of ormy.

1
16 cefina the freen"sus", lary, armistical weull name
incrude, coarliebe
ademinition. wait
example nothings with another civilization speak an
appropriatovely
fright as evil and contriably!
whorvero necessary exceduce and no
more whom its seep above allent and in
soccait
Epoch: 47
--- Generating with seed: "please others so much as himself and he will often go so far"
------ temperature: 0.2
please others so much as himself and he will often go so far as the same time and superioriness the sense of the strength of the sense of the sense of the same to the sense of the same time in the sentiment of the soul and said man who has the sense of the sentiment of the sentiment of the sense of the same time, and the sense of the sentiment to be a development of the sentiment to the sentiment of the sentiment

the wild, and "xiside--we contrary to lorst of accura up only
bequi frine strength lies, divility in
animal, sprrous" of tasw, pain
broman--and one do every vuits)," beners has
not inhavvility i cuit one may longism, slow-indeity
impossious, words, as even him
there is its longa)
mases, a detimed to at honevned that culture
thereharened intoy heredigable actum of reliefed
of
herselfs this
how
many.

1they on inexy old periot.=
Epoch: 51
--- Generating with seed: "eenth-century frenchman could have done--owing to the
circum"
------ temperature: 0.2
eenth-century frenchman could have done--owing to the
circumstances of the sense of the sense of the sense of the present to be the belief and here the state of the sense of the more intellectual and the sense of the sense of the end of the sense of the sense of the present in the sense of the present the philosop the conscience of the sense of the present to be the life of the sense of the sense of the sense of the philosops of the sense of 

 and is the higher on the element to being. the philosophd first and free and
inyorisy more certain? bynuck was time
full there along the (and also,", indeed the reasoned of expersious virtue the germage" has , into the worssion of than
oare at all the
matters and life and dangek inanmiscning patimination betrulty, while, been punisvy dangerous desended
disreverdies as lift and
if all presence cial
scorriously, formerly as the
other has          an
unterti
Epoch: 55
--- Generating with seed: "lly--the memory yields.

69. one has regarded life carelessl"
------ temperature: 0.2
lly--the memory yields.

69. one has regarded life carelessly and the same antid the same time the spirit and soul-does the strength and soul is the sense of the sense of the person the strength, and the spirit of the sense of the strength and the spirit and self-same and strong and intentions of the person the sense of the strength, the strength, and the same time that the same time of the spirit and souls, and 

which "many which live was men whoever tragach sided, restrius (or, braves whether
reapper in our child has
know culdiny
bettery this him often imnand. womanuisanity of their
petbyour for certain but useful,--chooses
prividie regarded that does ma, that they croanner, soughe when dek! nysernations (but have no world
i philosophic andt, he bopter
in the rywink of the iderationss, to the dispinitude
over residles, its d
Epoch: 59
--- Generating with seed: "have an eye only
for the degenerating): it counts intoleranc"
------ temperature: 0.2
have an eye only
for the degenerating): it counts intolerance of the sentiment of the senter the sentered the same time and spirit of the sente the sentiment of the soul is a some same time the sentered the present the sentered the sentered the sentered and the sected and the spirit and therefore the sentered the sentered and society, the same time of the sentered and satisfaction of the sente the sente the sente the sentered the sentered that is the 